In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import os
import seaborn
from tqdm import tqdm



trainPath = 'data/Dataset_1_train_asset.csv'
targetPath = 'data/Dataset_1_train_payoff.csv'

trainData = pd.read_csv(trainPath)
target = pd.read_csv(targetPath)

In [ ]:
# exercise 1
import numpy as np


N = norm.cdf

def bs_call(K: float, 
            T: float, 
            S: float = 1.0, 
            r: float = 0.0, 
            sigma: float = 15.8):
    
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T) * N(d2)

def bs_put(K: float,
           T: float,
           S: float = 1.0,
           r: float = 0.0,
           sigma: float = 15.8):
    
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)